In [1]:
import numpy as np
import pandas as pd

In [2]:
# Generate example data
date_range = pd.date_range(start='2024-01-01', periods=2_000, freq='d')
data = np.random.normal(50, 8, 2000)
df = pd.DataFrame(data, index=date_range, columns=['value'])

In [3]:
df.head()

,value
2024-01-01,37.965522
2024-01-02,42.725369
2024-01-03,53.257722
2024-01-04,67.104064
2024-01-05,57.242365


In [4]:
from spyc import SPC, x_chart, mr_chart

In [5]:
spc_example = SPC(data_in=df, 
                  target_col = 'value',
                  # Default values to None (will use all available data for control line calc.)
                  fix_control_start_dt=None,
                  fix_control_end_dt=None,
                  # Data shows repeated patterns every 7 days
                  seasonal=7
                 )

In [6]:
# OPTIONAL. i.e., new beds added and control lines need to be re-calculated
# (can be done once, or multiple times)
spc_example.add_process_change_date(change_date='2025-01-21',
                                    fix_control_start_dt='2025-01-27',
                                    fix_control_end_dt='2028-09-27',
                                    process_name='New ward')

In [7]:
# SPC data/chart stored for specific SPC function. The function can be custom/use built in 
# but requires a dictionary with 'prococes', 'CL', 'LCL', 'UCL' to be returned

def quantile_chart_calc(data, target_col, cl_start_dt=None, cl_end_dt=None):
    
    CL = data[target_col].quantile(0.5)
    LCL = data[target_col].quantile(0.01)
    UCL = data[target_col].quantile(0.99)
    
    return {'process': data[target_col], 'CL': CL, 'LCL': LCL, 'UCL': UCL}
    
# Specific rules can then be defined for each chart type
spc_example.add_spc(spc_name='Custom quantile-chart test', 
                    spc_calc_func=quantile_chart_calc, 
                    rule_1=True) 

# Add in X and mR chart for comparison (built in functions)
spc_example.add_spc(spc_name='X-chart', 
                    spc_calc_func=x_chart, 
                    rule_1=True, 
                    rule_3 = True)
spc_example.add_spc(spc_name='mR-chart', 
                    spc_calc_func=mr_chart, 
                    rule_1=True)

In [8]:
spc_example.spc_outputs_dict['Custom quantile-chart test']['data']

,CL,LCL,UCL,process,period,season,Rule 1 violation
2024-01-01,50.900186,36.191907,64.526245,37.965522,0.0,0,0
2024-01-02,49.217899,33.768982,67.117690,42.725369,0.0,1,0
2024-01-03,49.846401,34.793495,63.437581,53.257722,0.0,2,0
2024-01-04,52.139063,36.886234,68.992434,67.104064,0.0,3,0
2024-01-05,49.959746,38.584066,66.955410,57.242365,0.0,4,0
...,...,...,...,...,...,...,...
2029-06-18,50.180365,34.935907,68.510687,52.228263,1.0,0,0
2029-06-19,49.707455,32.525031,68.417551,52.382076,1.0,1,0
2029-06-20,50.354206,31.239840,67.694490,49.927098,1.0,2,0
2029-06-21,49.238449,28.814112,68.092399,41.607156,1.0,3,0


In [10]:
spc_example.plot_spc(file_name='spc_chart_test', save_to_html=True)